In [78]:
import pandas as pd
import json
from tqdm import tqdm
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
import stopwordsiso as stopwords
import pickle
class Languages:
    esga = 'ES-GA'
    si = 'SI'
    gb = 'GB'
    hu = 'HU'
    ua = 'UA'
    all = [esga, si, gb, hu, ua]

In [18]:
sentences = {}
for language in Languages.all:
    print('loading', language)
    sentences[language] = pd.read_feather(f'artefacts/pandas_frames/{language}_parlamint_sentences.feather')
metadata = pd.read_feather('artefacts/pandas_frames/parlamint_metadata.feather')

In [3]:
metadata

,index,ID,Title,Date,Body,Term,Session,Meeting,Sitting,Agenda,Subcorpus,Speaker_role,Speaker_MP,Speaker_Minister,Speaker_party,Speaker_party_name,Party_status,Speaker_name,Speaker_gender,Speaker_birth
0,0,ParlaMint-ES-GA_2017-09-26-DSPG040.u1,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
1,1,ParlaMint-ES-GA_2017-09-26-DSPG040.u2,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
2,2,ParlaMint-ES-GA_2017-09-26-DSPG040.u3,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Regular,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Arias Rodríguez, Raquel",F,1966
3,3,ParlaMint-ES-GA_2017-09-26-DSPG040.u4,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
4,4,ParlaMint-ES-GA_2017-09-26-DSPG040.u5,Minutes of the Assembly of the Galician Parlia...,2017-09-26,Unicameralism,010,,040,2017-09-26,,Reference,Chairperson,MP,-,PPdeG,Partido Popular de Galicia,Coalition,"Santalices Vieira, Miguel Ángel",M,1955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1365545,213,ParlaMint-UA_2019-11-13-m0.u214,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Regular,MP,-,фЄС,"Фракція політичної партії ""Європейська солідар...",Opposition,"Олексійович Гончаренко, Олексій",M,1980
1365546,214,ParlaMint-UA_2019-11-13-m0.u215,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Chairperson,MP,-,,,,"Олександрович Разумков, Дмитро",M,1983
1365547,215,ParlaMint-UA_2019-11-13-m0.u216,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Regular,MP,-,фЄС,"Фракція політичної партії ""Європейська солідар...",Opposition,"Володимирівна Геращенко, Ірина",F,1971
1365548,216,ParlaMint-UA_2019-11-13-m0.u217,"Ukrainian parliamentary corpus ParlaMint-UA, t...",2019-11-13,Unicameralism,9,2,,2019-11-13,,Reference,Chairperson,MP,-,,,,"Олександрович Разумков, Дмитро",M,1983


In [82]:
sentences[Languages.si]

,sentence_id,speech_id,language,text,lemmas
0,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.ana.se...,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.u1,SI,Spoštovane kolegice poslanke in kolegi poslanc...,spoštovan kolegica poslanka in kolega poslanec...
1,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.ana.se...,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.u1,SI,"Začenjam 8. izredno sejo Državnega zbora, ki s...",začenjati izreden seja državen zbor ki biti on...
2,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.ana.se...,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.u1,SI,"Obveščen sem, da se današnje seje ne more udel...",obveščen biti da se današnji seja ne moči udel...
3,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.ana.se...,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.u1,SI,Vse prisotne lepo pozdravljam.,ves prisoten lepo pozdravljati
4,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.ana.se...,ParlaMint-SI_2014-12-23-SDZ7-Izredna-08.u1,SI,Prehajamo na določitev dnevnega reda 8. izredn...,prehajati na določitev dneven red izreden seja...
...,...,...,...,...,...
3876178,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.ana.seg8...,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.u170,SI,S tem prekinjam to točko dnevnega reda.,z ta prekinjati ta točka dneven red
3876179,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.ana.seg8...,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.u170,SI,"Prekinjam pa tudi 5. sejo zbora, ki jo bomo na...",prekinjati pa tudi seja zbor ki on biti nadalj...
3876180,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.ana.seg8...,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.u170,SI,Lep večer še naprej vsem!,lep večer še naprej ves
3876181,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.ana.seg8...,ParlaMint-SI_2019-03-05-SDZ8-Redna-05.u170,SI,( SEJA JE BILA PREKINJENA 5.,seja biti biti prekinjen


In [36]:
for language in tqdm(Languages.all):
    sentences[language].speech_id = sentences[language].speech_id.apply(lambda x: x.replace('.ana', ''))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  2.27it/s]


In [43]:
speeches = {}
for language in tqdm(sentences):
    with open(f'artefacts/pandas_frames/speech2lemmas_{language}.pkl', 'rb') as f:
        s2l = pickle.load(f)
        for s in list(s2l.keys()):
            s2l[s.replace('.ana', '')] = s2l[s]
            s2l.pop(s)
        speeches[language] = s2l

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:18<00:00,  3.74s/it]


## Prepare data for TF-IDF

### Time period
Based on Bojan's sentiment scores

In [7]:
sentences_gb = sentences[Languages.gb][sentences[Languages.gb].speech_id.str.contains('commons')]
metadata_gb = metadata[metadata.ID.isin(sentences_gb.speech_id.unique())]

In [9]:
may_1_to_32_2020_metadata = metadata_gb[(metadata_gb.Date >= '2020-05-01') & (metadata_gb.Date <= '2020-05-31')]
may_1_to_32_2020_sentences = sentences_gb[sentences_gb.speech_id.isin(may_1_to_32_2020_metadata.ID)]

In [10]:
con_sentences = may_1_to_32_2020_sentences[may_1_to_32_2020_sentences.speech_id.isin(
    may_1_to_32_2020_metadata[may_1_to_32_2020_metadata.Speaker_party == 'CON'].ID.unique())]
lab_sentences = may_1_to_32_2020_sentences[may_1_to_32_2020_sentences.speech_id.isin(
    may_1_to_32_2020_metadata[may_1_to_32_2020_metadata.Speaker_party == 'LAB'].ID.unique())]
'con:', len(con_sentences), 'lab:', len(lab_sentences)

('con:', 10021, 'lab:', 4672)

In [11]:
# get lemma groups by speach_id
lemmas = [
    ('con', ' '.join(con_sentences.groupby('speech_id').lemmas.apply(lambda x: ' '.join(x)))),
    ('lab', ' '.join(lab_sentences.groupby('speech_id').lemmas.apply(lambda x: ' '.join(x)))),
]

### By list of speeches

In [67]:
lemmas = []
for speaker_name, speech_ids in tqdm(metadata[metadata.language == Languages.si].groupby('Speaker_name').ID):
    lemmas.append((
        speaker_name,
        ' '.join([speeches[Languages.si][sid] for sid in speech_ids.values if sid in speeches[Languages.si]])
    ))
lemmas = list(sorted(lemmas, key=lambda x: len(x[1])))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 973/973 [00:00<00:00, 2372.28it/s]


#### Limit speakers

In [73]:
lemmas = []
speakers = ['Janša, Ivan Janez', 'Janković, Zoran']
for speaker_name, speech_ids in tqdm(metadata[(metadata.language == Languages.si) & (metadata.isin(speakers).Speaker_name)].groupby('Speaker_name').ID):
    lemmas.append((
        speaker_name,
        ' '.join([speeches[Languages.si][sid] for sid in speech_ids.values if sid in speeches[Languages.si]])
    ))
lemmas = list(sorted(lemmas, key=lambda x: len(x[1])))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 483.08it/s]


### Topics

In [115]:
lemmas = []
topic = 'healthcare'
artefacts_by_topic = Path('artefacts/by_topic')
healthcare = pd.read_feather(artefacts_by_topic / 'healthcare.feather')
healthcare.count()
speech2lemmas = sentences[Languages.si].groupby('speech_id').lemmas.apply(lambda x: ' '.join(x))

for party, party_metadata in tqdm(metadata[metadata.language == Languages.si].groupby('Speaker_party')):
    party = party if party.strip() else '/'
    ids = [i for i in party_metadata.ID if i in speech2lemmas]
    lemmas.append(
        (party, ' '.join(speech2lemmas[ids]))
    )
lemmas = list(sorted(lemmas, key=lambda x: len(x[1]), reverse=True))
rows = []
for party, party_lemmas in lemmas:
    rows.append([party, len(party_lemmas)])
pd.DataFrame(rows, columns=['party', 'char count'])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 31.34it/s]


,party,char count
0,SDS,100707360
1,SD,50113142
2,/,46858523
3,LDS,34122868
4,NSi,32574398
5,SMC,24639334
6,DeSUS,22418047
7,SNS,18254614
8,SLS,17299728
9,Levica,12044309


## Calculate TF-IDF

In [123]:
sw = stopwords.stopwords('sl')
print(len(sw))
vectorizer = TfidfVectorizer(stop_words=list(sw), min_df=6, max_df=0.7)

documents = [l for _, l in lemmas]
X = vectorizer.fit_transform(documents)

tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

tfidf_df = tfidf_df.T
top_words = {}
for i in range(len(documents)):
    sorted_df = tfidf_df[i].sort_values(ascending=False)
    top_words[lemmas[i][0]] = sorted_df.index.tolist()
    top_words[f'{lemmas[i][0]}_scores'] = sorted_df.tolist()
tf_idf = pd.DataFrame(top_words)

446


,SDS,SDS_scores,SD,SD_scores,/,/_scores,LDS,LDS_scores,NSi,NSi_scores,...,SLS+SKD,SLS+SKD_scores,Lipa,Lipa_scores,ZaAB,ZaAB_scores,Konkretno,Konkretno_scores,DLGV,DLGV_scores
0,cvikl,0.260667,trček,0.153028,covid,0.347647,sok,0.461080,covid,0.218410,...,skd,0.420937,lipa,0.895809,dutb,0.314919,covid,0.455649,gašpar,0.407305
1,smc,0.254095,smc,0.137306,sdh,0.265614,peče,0.242579,dutb,0.215155,...,krma,0.284164,zagorac,0.158828,sdh,0.287535,vesoljski,0.449037,multipleks,0.399287
2,šarec,0.177262,covid,0.136321,dutb,0.237964,barovič,0.205041,cvikl,0.182055,...,sok,0.249551,paloma,0.146449,vajeništvo,0.170691,krvavec,0.193896,frangež,0.309448
3,potrč,0.164537,žibert,0.127596,terminal,0.145795,kelemina,0.200825,sib,0.160552,...,anderlič,0.185713,cvikl,0.136706,bla,0.163584,pandemija,0.193078,mišič,0.205546
4,anderlič,0.156302,kordiš,0.127231,narodnostno,0.121999,rokavec,0.200626,migrantski,0.157700,...,potrč,0.179909,tavš,0.102073,vogrin,0.152549,valuten,0.163373,jakič,0.200898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19879,prevalenca,0.000000,borštnikov,0.000000,cveto,0.000000,odgovora,0.000000,okorel,0.000000,...,masaža,0.000000,marcel,0.000000,malni,0.000000,madžarščina,0.000000,mafija,0.000000
19880,metodičen,0.000000,graščak,0.000000,neugotovljen,0.000000,odhiteti,0.000000,trpati,0.000000,...,masaker,0.000000,maratonski,0.000000,maline,0.000000,madžarka,0.000000,madžarščina,0.000000
19881,edo,0.000000,grebsti,0.000000,kosova,0.000000,odhoden,0.000000,okončina,0.000000,...,marža,0.000000,maratonec,0.000000,malina,0.000000,madridski,0.000000,madžarka,0.000000
19882,mevža,0.000000,borilen,0.000000,kosovel,0.000000,tovarištvo,0.000000,okoličan,0.000000,...,maršal,0.000000,maraton,0.000000,maligen,0.000000,madrid,0.000000,madridski,0.000000
